In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score,classification_report,roc_curve,roc_auc_score,confusion_matrix,precision_recall_curve,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,cross_val_predict
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder,FunctionTransformer
from imblearn.under_sampling import RandomUnderSampler,NearMiss,TomekLinks,InstanceHardnessThreshold
from sklearn.compose import ColumnTransformer
import optuna
from sklearn.metrics import average_precision_score
from sklearn.utils import shuffle
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE,ADASYN
from sklearn.metrics import average_precision_score

In [3]:
df = pd.read_csv('fraud.csv')

In [4]:
df.shape

(6362620, 11)

In [5]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
df = df[~(df['nameDest'].str.startswith('M'))|(df['nameOrig'].str.startswith('M'))]

In [7]:
df.shape

(4211125, 11)

In [8]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.00,1,0
9,1,DEBIT,5337.77,C712410124,41720.0,36382.23,C195600860,41898.0,40348.79,0,0
10,1,DEBIT,9644.94,C1900366749,4465.0,0.00,C997608398,10845.0,157982.12,0,0
15,1,CASH_OUT,229133.94,C905080434,15325.0,0.00,C476402209,5083.0,51513.44,0,0


In [9]:
df.drop(columns=['nameOrig','nameDest'],inplace=True)

In [10]:
df.head(2)

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.0,181.0,0.0,0.0,0.0,1,0
3,1,CASH_OUT,181.0,181.0,0.0,21182.0,0.0,1,0


In [11]:
df['diffOrg'] = df['oldbalanceOrg'] - df['newbalanceOrig']
df['diffdest'] = df['oldbalanceDest'] - df['newbalanceDest']
df['isFraud'] = df['isFraud'].astype('int')
df.drop(columns=['oldbalanceOrg','newbalanceOrig','newbalanceDest','oldbalanceDest'],inplace=True)
hours = df['step'] % 24
bins = [0, 6, 12, 18, 24]
labels = ['Late Night', 'Morning', 'Afternoon', 'Night']

buckets =  pd.cut(
    hours,
    bins=bins,
    labels=labels,
    right=False  # 0-5, 6-11, 12-17, 18-23
)
df['transaction_time'] = buckets

In [12]:
df.drop(columns=['step'],inplace=True)

In [13]:
df.drop(columns=['isFlaggedFraud'],inplace=True)

In [14]:
df.head()

,type,amount,isFraud,diffOrg,diffdest,transaction_time
2,TRANSFER,181.00,1,181.00,0.00,Late Night
3,CASH_OUT,181.00,1,181.00,21182.00,Late Night
9,DEBIT,5337.77,0,5337.77,1549.21,Late Night
10,DEBIT,9644.94,0,4465.00,-147137.12,Late Night
15,CASH_OUT,229133.94,0,15325.00,-46430.44,Late Night


In [15]:
# Create train , test , validation Dataset
train,val = train_test_split(df,stratify=df['isFraud'],test_size=0.1,random_state=42)

In [16]:
train['isFraud'].value_counts()/train.shape[0]

isFraud
0    0.99805
1    0.00195
Name: count, dtype: float64

In [17]:
val['isFraud'].value_counts()/val.shape[0]

isFraud
0    0.99805
1    0.00195
Name: count, dtype: float64

In [17]:
# Save Validation data
#val.to_csv('validation_set.csv',index=False)

In [18]:
# using 25% random non_fraud data as for sample
fraud_df = train[train['isFraud']==1]
non_fraud_df = train[train['isFraud']==False].sample(int(len(train) * 0.50),random_state=42)
sample_df = pd.concat([fraud_df, non_fraud_df])
sampel_df = sample_df.sample(len(sample_df))
sample_df.shape

(1902398, 6)

In [19]:
x,y = sample_df.drop(columns=['isFraud']),sample_df['isFraud']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [20]:
# log transformer
log_transformer = FunctionTransformer(
    lambda x: np.log1p(np.abs(x)),
    feature_names_out="one-to-one"
)

In [21]:
# preprocessor
preprocessor = ColumnTransformer(transformers=[
                                              ('num',log_transformer,['amount','diffOrg','diffdest']),
                                              ('cat',OneHotEncoder(),['type'])
                                              ])


In [22]:
# create model
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()

# scale_pos_weight = neg / pos
sampler = SMOTE(sampling_strategy=0.2, random_state=42)
lgbm = LGBMClassifier(random_state=42)
pipeline = Pipeline([
      ('preprocessing',preprocessor),
      ('sampling',sampler),
      ('model',lgbm)
])

In [23]:
pipeline.fit(x_train,y_train)
y_pred = pipeline.predict(x_test)
accuracy_score(y_test,y_pred)

[LightGBM] [Info] Number of positive: 303195, number of negative: 1515976
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009457 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 823
[LightGBM] [Info] Number of data points in the train set: 1819171, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609439
[LightGBM] [Info] Start training from score -1.609439


C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


0.9885670731707317

In [25]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99    379030
           1       0.23      0.84      0.36      1450

    accuracy                           0.99    380480
   macro avg       0.61      0.91      0.68    380480
weighted avg       1.00      0.99      0.99    380480



In [26]:
print(confusion_matrix(y_test,y_pred))

[[374911   4119]
 [   231   1219]]


In [28]:
y_prob = pipeline.predict_proba(x_test)[:,1]

precision,recall,pr_thresold = precision_recall_curve(y_test,y_prob)

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [29]:
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-12)

In [30]:
best_idx = np.nanargmax(f1_scores[:-1]) # exclude last element as it corresponds to threshold=none
best_threshold = pr_thresold[best_idx]
best_precision = precision[best_idx]
best_recall = recall[best_idx]
best_f1 = f1_scores[best_idx]
print(f"Best threshold by F1: {best_threshold:.4f}")
print(f"Precision={best_precision:.4f}, Recall={best_recall:.4f}, F1={best_f1:.4f}")

Best threshold by F1: 0.9812
Precision=0.9684, Recall=0.7186, F1=0.8250


# Tunning

In [38]:
def objective(trial):
  # Define Parameters
  params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 120),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'min_child_samples':  trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_estimators': trial.suggest_int('n_estimators',100,1000,step=100),
        'n_jobs': -1,
        'random_state': 42
        
    }

    

  log_transformer = FunctionTransformer(
    lambda x: np.log1p(np.abs(x)),
    feature_names_out="one-to-one"
    )

  # Column transformer
  preprocessor = ColumnTransformer(transformers=[
                                              ('num',log_transformer,['amount','diffOrg','diffdest']),
                                              ('cat',OneHotEncoder(),['type','transaction_time'])
                                              ])


  # create model
  lgbm = LGBMClassifier(**params)
  pipeline = Pipeline([
      ('preprocessing',preprocessor),
      ('sampling',SMOTE(sampling_strategy=0.2, random_state=42)),
      ('model',lgbm)
  ])

  cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

  y_pred_proba = cross_val_predict(pipeline,x_train,y_train,cv=cv,method='predict_proba')[:,1]
   
  # Find best threshold INSIDE CV
  precision, recall, thresholds = precision_recall_curve(y_train, y_pred_proba)
  f1_scores = 2 * precision * recall / (precision + recall + 1e-12)

  best_f1 = np.max(f1_scores[:-1])  # ignore last threshold
  return best_f1


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=30,show_progress_bar=False)

[I 2026-01-04 12:09:07,865] A new study created in memory with name: no-name-1abbb0f0-c1a5-4cc5-8cd4-243037045e7b


[LightGBM] [Info] Number of positive: 202130, number of negative: 1010650
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.040651 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1310
[LightGBM] [Info] Number of data points in the train set: 1212780, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 202130, number of negative: 1010651
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041743 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1040
[LightGBM] [Info] Number of data points in the train set: 1212781, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609439
[LightGBM] [Info] Start training from score -1.609439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

[LightGBM] [Info] Number of positive: 202130, number of negative: 1010651
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057131 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1041
[LightGBM] [Info] Number of data points in the train set: 1212781, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609439
[LightGBM] [Info] Start training from score -1.609439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2026-01-04 12:54:13,155] Trial 27 finished with value: 0.843076343781866 and parameters: {'learning_rate': 0.14463340672888714, 'num_leaves': 86, 'max_depth': 7, 'min_child_samples': 84, 'subsample': 0.4937359851590104, 'colsample_bytree': 0.8301130533339118, 'reg_alpha': 3.980713734081538, 'reg_lambda': 3.6578052041726288, 'n_estimators': 200}. Best is trial 7 with value: 0.843454664545216.


[LightGBM] [Info] Number of positive: 202130, number of negative: 1010650
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049993 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1310
[LightGBM] [Info] Number of data points in the train set: 1212780, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 202130, number of negative: 1010651
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.048116 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1040
[LightGBM] [Info] Number of data points in the train set: 1212781, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609439
[LightGBM] [Info] Start training from score -1.609439


In [149]:
print(study.best_params)
print(study.best_value)

{'learning_rate': 0.056870550481473985, 'num_leaves': 49, 'max_depth': 4, 'min_child_samples': 86, 'subsample': 0.3405305779866419, 'colsample_bytree': 0.7669391171858071, 'reg_alpha': 1.102479528599375, 'reg_lambda': 1.2334000948279056, 'n_estimators': 400}
0.8534142900533228


# Model With Best Parameteres

In [31]:
# create model
best_params = {'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': 0.056870550481473985, 'num_leaves': 49, 'max_depth': 4, 'min_child_samples': 86, 'subsample': 0.3405305779866419, 
        'colsample_bytree': 0.7669391171858071, 'reg_alpha': 1.102479528599375,
        'reg_lambda': 1.2334000948279056, 'n_estimators': 400,
        'n_jobs': -1,'is_unbalance': True}

sampler = SMOTE(sampling_strategy=0.2, random_state=42)
lgbm = LGBMClassifier(**best_params,random_state=42)
pipeline = Pipeline([
      ('preprocessing',preprocessor),
      ('sampling',sampler),
      ('model',lgbm)
])

pipeline.fit(x_train,y_train)
y_pred = pipeline.predict(x_test)
print(accuracy_score(y_test,y_pred))

[LightGBM] [Info] Number of positive: 303195, number of negative: 1515976
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017314 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 823
[LightGBM] [Info] Number of data points in the train set: 1819171, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609439
[LightGBM] [Info] Start training from score -1.609439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


0.9400809503784693


In [32]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97    379030
           1       0.06      0.99      0.11      1450

    accuracy                           0.94    380480
   macro avg       0.53      0.96      0.54    380480
weighted avg       1.00      0.94      0.97    380480



In [33]:
y_prob = pipeline.predict_proba(x_test)[:,1]

precision,recall,pr_thresold = precision_recall_curve(y_test,y_prob)

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [34]:
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-12)

In [35]:
best_idx = np.nanargmax(f1_scores[:-1]) # exclude last element as it corresponds to threshold=none
best_threshold = pr_thresold[best_idx]
best_precision = precision[best_idx]
best_recall = recall[best_idx]
best_f1 = f1_scores[best_idx]
print(f"Best threshold by F1: {best_threshold:.4f}")
print(f"Precision={best_precision:.4f}, Recall={best_recall:.4f}, F1={best_f1:.4f}")

Best threshold by F1: 0.9942
Precision=0.9784, Recall=0.7186, F1=0.8286
